<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [ ]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

## Step 2 - Loading Data & Preprocessing

In [ ]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)

In [ ]:
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

In [ ]:
train_batch = train_data_iterator.next()
test_batch = test_data_iterator.next()

In [ ]:
test_batch

In [ ]:
len(test_batch)

In [ ]:
train_batch[0].shape

In [ ]:
train_data = train_data.map(lambda x,y: (x/255,y))
test_data = test_data.map(lambda x,y: (x/255,y))

In [ ]:

scaled_train_iterator = train_data.as_numpy_iterator()
scaled_test_iterator = test_data.as_numpy_iterator()
train_batch = scaled_train_iterator.next()
test_batch = scaled_test_iterator.next()
test_batch[0].max()

In [ ]:
len(train_data)

In [ ]:
train_size = int(len(train_data)*0.8)
val_size = int(len(train_data)*0.2)+1
test_size = len(test_data)
train_size+val_size

In [ ]:
train = train_data.take(train_size)
val = train_data.skip(train_size).take(val_size)
test = test_data.take(test_size)

In [ ]:
print(len(train),len(val),len(test))

In [ ]:
train_images, train_labels = [], []
for image, label in train:
    train_images.append(image)
    train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

val_images, val_labels = [], []
for image, label in val:
    val_images.append(image)
    val_labels.append(label)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

In [ ]:
data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [ ]:
batch_size = 32
train_generator = data_augmentation.flow(
    train,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
val_generator = train_data.flow(
    val,
    batch_size=batch_size,
    shuffle=False
)

## Step 3 - Model

In [ ]:
model = Sequential(
    [
        Conv2D(32, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(64, (3,3), 1,  activation = 'relu'),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(16, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),
        Dropout(0.25),

        Flatten(),

        Dense(256, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [ ]:
model.compile(
    optimizer = Adam(learning_rate = 0.07),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [ ]:
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, monitor='val_loss', min_lr=0.00001)

In [ ]:
history = model.fit(train, validation_data = val, epochs = 20, callbacks = [tensorboard_callback, ])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


635/635 [==============================] - ETA: 0s - loss: 409.7466 - accuracy: 0.2447

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/gdrive/MyDrive/Human emotions/Saved_model')


In [ ]:

loaded_model = load_model('/content/gdrive/MyDrive/Human emotions/Saved_model')


In [ ]:
predictions = loaded_model.predict(test)

118/231 [==============>...............] - ETA: 8:21

KeyboardInterrupt: ignored

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
test_labels = np.concatenate([y for x, y in test], axis=0)

In [ ]:
correct_predictions = np.equal(predicted_classes, test_labels)

In [ ]:
accuracy = np.mean(correct_predictions)
print("Test Accuracy:", accuracy)